# Preparação da base para leitura do Dashboard
Algumas transformações precisam ser realizadas para exibição no dashbard.
1. Limpeza e carga das colunas NEGATIVO, POSTIVO, NEUTRO e SPAM
2. Tratamento para nuvem de palavras

In [104]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import unidecode
import itertools
import collections

In [105]:
#carregando 
df = pd.read_pickle('dados_twitter_ans_classificados_analisados.pkl')

#limpar colunas
df['POSITIVO'] = 0
df['NEGATIVO'] = 0
df['NEUTRO'] = 0
df['SPAM'] = 0

In [106]:
#carregando 
df['POLARIDADE'] = ''
for index,row in df.iterrows():
    if row['valor_sentimento'] == 0:
        df.at[index,'NEUTRO'] = 1
        df.at[index,'POLARIDADE'] = 'NEUTRO'
    elif row['valor_sentimento'] > 0:
        df.at[index,'POSITIVO'] = 1
        df.at[index,'POLARIDADE'] = 'POSTIVO'
    elif row['valor_sentimento'] < 0:
        df.at[index,'NEGATIVO'] = 1
        df.at[index,'POLARIDADE'] = 'NEGATIVO'
        
#definindo a coluna index como str
df['id']=df['id'].astype(str)

### Preparando a nuvem de palavras
Será passado para minusculas e removido URL, usuários, HASHTAGS, stopwords

In [107]:
def remove_url(txt):
    
    return " ".join(re.sub(r'http\S+', '', txt, flags=re.MULTILINE).split())

In [108]:
def remove_users(txt):
    
    return " ".join(re.sub(r'@\S+',"",txt).split())

In [109]:
def remove_stopwords(text, *stopwords):

    return ' '.join([word for word in text.split() if word not in stopwords])

In [110]:
#aplicando minusculas
df["nuvem_palavras"] = df.text.apply(str.lower)

#adiconando a coluna text_tratada removendo as urls
df["nuvem_palavras"] = df.nuvem_palavras.apply(remove_url)
df["nuvem_palavras"] = df.nuvem_palavras.apply(remove_users)

df["nuvem_palavras"] = df['nuvem_palavras'].str.replace('[^\w\s]','')
df["nuvem_palavras"] = df['nuvem_palavras'].apply(unidecode.unidecode)

#baixando as stopwords
nltk.download('stopwords')
stop_words_acento = set(stopwords.words('portuguese'))
#retirando acentuacao
stop_words = set([unidecode.unidecode(word) for word in stop_words_acento])
#adicionando palavras que nao vieram nas stopwords
tirar_essas_tambem = set([
'e','a','pra','o','nao','dia','ta','dm','vc','ser','vcs','que','ai',
'q','vai','tar','ola','faz','fazer','ter','p','pq','to','ne'])
#retirando palavras
df["nuvem_palavras"] = df.nuvem_palavras.apply(remove_stopwords,args=stop_words)
df["nuvem_palavras"] = df.nuvem_palavras.apply(remove_stopwords,args=tirar_essas_tambem)

#colocando todas as palavras em uma única lista
palavras_tweets = [tweet.lower().split() for tweet in df['nuvem_palavras']]
todas_palavras = list(itertools.chain(*palavras_tweets))

#contando as palavras
frequencia_palavras = collections.Counter(todas_palavras)

[nltk_data] Downloading package stopwords to /Users/lups/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Classificando os usuários
Será criado um campo a mais para classificar o número quanto ao numero de usuários.

In [117]:
#criando novas colunas
df['USER_SEGUIDORES'] = ''
df['USER_SEGUIDORES_SEGUIDORES'] = ''
df['USER_SEGUIDORES_ALTA'] = 0
df['USER_SEGUIDORES_MEDIA'] = 0
df['USER_SEGUIDORES_BAIXA'] = 0
df['USER_SEGUIDORES_ALTO'] = 0
df['USER_SEGUIDORES_MEDIO'] = 0
df['USER_SEGUIDORES_BAIXO'] = 0
df['USER_SEGUIDORES_SINISTRO'] = 0
df['USER_SEGUIDORES_SINISTRA'] = 0

for index,row in df.iterrows():
    if row['followers_count'] in range(0,100):
        df.at[index,'USER_SEGUIDORES'] = 'BAIXA'
        df.at[index,'USER_SEGUIDORES_BAIXO'] = 1
    elif row['followers_count'] in range(101,1000):
        df.at[index,'USER_SEGUIDORES'] = 'MEDIA'
        df.at[index,'USER_SEGUIDORES_MEDIO'] = 1
    elif row['followers_count'] in range(1001,100000):
        df.at[index,'USER_SEGUIDORES'] = 'ALTA'
        df.at[index,'USER_SEGUIDORES_ALTO'] = 1
    elif row['followers_count'] > 100000:
        df.at[index,'USER_SEGUIDORES'] = 'SINISTRA'         
        df.at[index,'USER_SEGUIDORES_SINISTRO'] = 1


        

In [118]:
#writer = pd.ExcelWriter("tweets_ans.xlsx", datetime_format='mmm d yyyy hh:mm:ss')
df.to_excel("tweets_ans.xlsx")